In [1]:
from pylsl import StreamInlet, resolve_stream
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
import scipy.signal as sig
from scipy.fft import fft, ifft, fftfreq
from scipy.signal import find_peaks
from scipy import fft
from scipy import stats
from collections import Counter

from sklearn.decomposition import FastICA, PCA

In [13]:
channels = 8 # for this dataset
# sample rate and lower and upper bounds
fs = 250
df = 1/fs
lowcut = 5
highcut = 40
order = 6
num_channels = 8
num_channel_used = 1
num_commands = 2
# frequencies = [30, 15, 10, 6]
frequencies = [7, 12]
ch = 3
alpha_threshold = 180000

# Receive chunks in real time

In [ ]:
print('starting')
streams = resolve_stream('type','EEG')

inlet = StreamInlet(streams[0])
data = []

sample_count = 0
time = 0
commands = []
# to store the final command
command = 0
# set up pointers
ptr = 0
history_data = np.zeros((num_commands,5))

while True: # keep running
    chunk, timestamps = inlet.pull_chunk()
    
    if timestamps:
        data.extend(chunk)
        sample_count += len(timestamps)
        
    # every whatever block of timestamps (use 250 fs to calc 1s)
    # write a for loop
    if sample_count >= fs: 
        time += 1
        data_f = np.array(data)

        #### Preprocess with filters and ICA ####
        X = butterworth(data_f, lowcut, highcut, visualize=False)
#         X = ICA(X, visualize=False)   
        
        # output preprocessed data, call it channels (8 channels x T timesteps)
    
        #### Feature extraction ####
        data = X[ch] # 250 x 1
#         print('cleaned data per channel: ', data)
        data_fft = np.abs(np.fft.fft(data)) # 250 x 1
        freq = np.fft.fftfreq(len(data_fft), 1/fs)
        
        #### Get eyes opened (0) or closed (1)
        value = determine_eyes_closed(frequencies, freq, data_fft)
        
        if value == 0:
            print('EYES OPEN, 0')
        else:
            print('EYES CLOSED, 1')
            
        #### COMPLEX VERSION ####
        
#         # keep a 5s history of previous average intensities of each command/freq
#         command_at_t, history_data = calc_curr_intensity(frequencies, freq, data_fft, history_data, ptr, visualize=False)
        
#         # update ptr
#         if ptr < 4: 
#             ptr += 1
#         else: 
#             ptr = 0
        
#         # keep commands over the past five seconds
#         commands.append(command_at_t)
        
#         # determine command at every 5nth second by determining the majority of the command in the last 5 sec
#         if time%5==0:
#             print(commands)
#             commands = Counter(commands)
#             value, count = commands.most_common()[0]
#             command = value
            
#             print('Command at time ', time, ' is: ', command)
            
#             # reset commands list and final command
#             commands = []
#             command = 0
        
        data = []
        sample_count = 0

#     time = list(range(0,data_f.shape[0])) # making up x axis
        
#     if count > 500: # early stopping
#         break

starting
EYES OPEN, 0
EYES OPEN, 0
EYES OPEN, 0
EYES OPEN, 0
EYES OPEN, 0
EYES OPEN, 0
EYES OPEN, 0
EYES OPEN, 0
EYES OPEN, 0
EYES OPEN, 0
EYES OPEN, 0
EYES OPEN, 0
EYES OPEN, 0
EYES OPEN, 0
EYES OPEN, 0
EYES OPEN, 0
EYES OPEN, 0


# 

In [15]:
def butterworth(data, lowcut, highcut, visualize=False):
    
    sos = sig.butter(3, [lowcut*2/fs, highcut*2/fs], btype='bandpass', output='sos')

    filt_sigs = np.zeros_like(data)
    for i in range(num_channels):
        filt_sigs[:,i] = sig.sosfilt(sos, data[:,i])
        
    filt_sigs.shape
    X = filt_sigs.T
#     print(X.shape)
    
    if visualize:
        fig, ax = plt.subplots(2, 2, figsize=(14, 8))
        ax = ax.flatten()

        for i in range(num_channels):
            ax[i].set_title(f'Channel {i+1}')
            ax[i].plot(X[i], linewidth=0.6, color='k')
            ax[i].set_ylim((-np.mean(X[i]) - 3*np.std(X[i]), np.mean(X[i]) + 3*np.std(X[i])))

        fig.tight_layout()
        plt.show()
        
    return X

In [16]:
# ICA
def ICA(X, visualize=False):
    np.random.seed(0)
    ica = FastICA(n_components=channels)
    S_ = ica.fit_transform(X.T)
    U = S_.T 
    Winv = ica.mixing_
    
    if visualize:
        fig, ax = plt.subplots(2, 2, figsize=(14, 8))
        ax = ax.flatten()

        for i in range(channels):
            ax[i].set_title(f'IC {i+1}')
            ax[i].plot(U[i], linewidth=0.6, color='k')

        fig.tight_layout()
        plt.show()
        
    # find the peaks

    number_peaks_list = []

    for i in range(channels):
        number_peaks = 0
        rms = np.sqrt(sum([s**2 for s in U[i]])/len(U[i]))
        peak_indicies = find_peaks(U[i])[0]
#         print('peak indicies', peak_indicies)
#         for a in peak_indicies:
#             if U[i][a] > 3*rms:
#                 number_peaks += 1
#         number_peaks_list.append(number_peaks)
        number_peaks_list.append(len(peak_indicies))
#     print('number peaks list', number_peaks_list)

    #process by getting rid of channels above 3 std deviations of number of peaks
    std_dev_cutoff_mult = 1

    peaks_std = np.std(number_peaks_list)
    peaks_mean = np.mean(number_peaks_list)
    np.where(number_peaks_list > peaks_mean - std_dev_cutoff_mult*peaks_std, number_peaks_list, 0)
    good_components = np.nonzero(number_peaks_list)
#     print('number_peaks_list', number_peaks_list)
#     print('good components', good_components)
    
    Winv_0 = Winv[:,good_components]
    U_0 = U[good_components]
    X_cleaned = Winv_0 @ U_0
    
    if visualize:
        fig, ax = plt.subplots(8, 2, figsize=(14, 8))
        ax = ax.flatten()

        for i in range(8):
            ax[i].set_title(f'Channel {i+1}')
            ax[i].plot(X_cleaned[i], linewidth=0.6, color='k')

        fig.tight_layout()
        plt.show()
        
    return X_cleaned    

In [17]:
## Fourier Transform Get Average Intensity

def get_avg_intensity(current_data, freq, lo, hi):
    return np.mean(current_data[np.where(np.logical_and(freq >= lo, freq <= hi))])

def calc_curr_intensity(frequencies, freq, current_data, history_data, ptr, visualize=False):
    
    # Get this second's average intensity for the four commands/freqs
    
#     ThirtyHz = get_avg_intensity(current_data, freq, frequencies[0]-2, frequencies[0]+2)
#     FifteenHz  = get_avg_intensity(current_data, freq, frequencies[1]-2, frequencies[1]+2)
#     TenHz = get_avg_intensity(current_data, freq, frequencies[2]-2, frequencies[2]+2)
#     SixHz = get_avg_intensity(current_data, freq, frequencies[3]-2, frequencies[3]+2)
    
#     curr_int = [ThirtyHz, FifteenHz, TenHz, SixHz]
    curr_int = [TwentyHz, TenHz]
#     print('curr_int: ', curr_int)
    
    history_data[:,ptr] = curr_int
#     print('history_data: ', history_data)
    
    # To output the command at t=5n
#     # First method
    avg_intensity = np.mean(history_data, axis=1)
    print('avg_intensity: ', avg_intensity)
    
    command_at_all_t = np.argmax(avg_intensity)
    
    # Second method: average over Int_t-4 through Int_t-1 as Int_prev and calculate increase rate from InT_prev to InT_t
    # At t = 5n, choose the command with the majority vote to act at t=5n. if no majority, then wait for five more seconds

#     prev_int = np.hstack((history_data[:,:ptr], history_data[:,ptr+1:]))
#     avg_prev_int = np.mean(prev_int, axis=1)
#     increase_rate = []
    
#     for i in range(num_commands):
#         increase_rate.append((avg_prev_int[i] - curr_int[i])/avg_prev_int[i])

#     command_at_all_t = np.argmax(increase_rate)
    
#     print('command_at_t: ', command_at_all_t)
    
    if visualize:
        labels = ['20Hz', '10Hz']
        x = np.arange(len(labels))
        avg_freq = [TwentyHz, TenHz]
        plt.figure(figsize=(8, 8))
        plt.bar(x, avg_freq)
        plt.ylim([0, 2500])
        plt.xticks(x, labels=labels)
        plt.title('Frequency domain plot')
        plt.ylabel('Avg Intensity')
    
    return command_at_all_t, history_data
    

In [7]:
## RIP code

# Compare the current avg int with the previous 5 avg int when sample >= 5.
    # Pick the command with the highest percentage increase every 5 seconds

def determine_command_1(history_data):
    # First method: average over Int_t-4 through Int_t and select highest averaged intensity at each time point t = 5n
    # only call if t=5n, single decisive command
    command_at_t5n = np.argmax(np.mean(history_data, axis=1))
    return command_at_t5n
    
def determine_command_2(history_data, ptr):
    # Second method: average over Int_t-4 through Int_t-1 as Int_prev and calculate increase rate from InT_prev to InT_t
    # At t = 5n, choose the command with the majority vote to act at t=5n. if no majority, then wait for five more seconds
    curr_int = history_data[:,ptr]
    prev_int = np.hstack((history_data[:,:ptr], history_data[:,ptr+1:]))
    avg_prev_int = np.mean(prev_int, axis=1)
    increase_rate = []
    for i in range(4):
        increase_rate.append((avg_prev_int[i] - curr_int[i])/avg_prev_int[i])
    command_at_all_t = np.argmax(increase_rate)
#     print('command_at_t: ', command_at_all_t)
    return command_at_all_t
    

In [18]:
def determine_eyes_closed(frequencies, freq, current_data):
#     print('current data: ', current_data)
    alpha = get_avg_intensity(current_data, freq, frequencies[0], frequencies[1])
#     print('alpha: ', alpha)
    
    if alpha > alpha_threshold:
        return 1 # eyes closed
    else:
        return 0 # eyes open